In [29]:

import cv2
import math
import numpy as np
from PIL import Image
from scipy.linalg import lu

BLOCKSIZE = 2 # Kích thước các block
WM_SIZE = 32  # Kích thước dấu thủy vân
q = 10  # vòng lặp T cho arnold
T =  0.0275

In [30]:
def cholesky(A):
  # Kiểm tra xác định dương
  if (np.linalg.eigvals(A) > 0).all() == False:
    return False
  n = A.shape[0]  # Lấy kích thước của ma trận A
  for i in range(0, n):
    for j in range(0, n):
      # Kiểm tra cac phần tử hiện tại và chuyển vị của nó
      if A[i][j] != A[j][i]:
        return False # Trả về False nếu tồn tại bất kỳ cặp không đối xứng
  #print("Ma trận đối xứng và xác định dương")
  return True

In [31]:
# Khai báo ma trận
matrix = np.array([[215, 215, 214, 213],
                [215, 214, 213, 213],
                [214, 213, 213, 213],
                [213, 213, 213, 213]])


In [32]:
def arnold_transform(matrix, iterations):
    height, width = matrix.shape
    print(height, width)
    new_matrix = np.zeros((height, width), dtype=matrix.dtype)

    for _ in range(iterations):
        for x in range(height):
            for y in range(width):
                s = (x + y) % height
                t = (x + 2*y) % width
                new_matrix[s, t] = matrix[x, y]

        matrix = new_matrix.copy()

    return new_matrix

In [33]:
def invert_arnold_transform(matrix, iterations):
    height, width = matrix.shape
    print(height, width)
    new_matrix = np.zeros((height, width), dtype=matrix.dtype)

    for _ in range(iterations):
        for s in range(height):
            for t in range(width):
                x = (2*s - t) % height
                y = (t - s) % width
                new_matrix[x, y] = matrix[s, t]

        matrix = new_matrix.copy()
    return new_matrix

In [34]:
watermark = cv2.imread("/content/w_binary.png", cv2.IMREAD_GRAYSCALE)
arnold_watermark = arnold_transform(watermark, 10)
cv2.imwrite("arnold_watermark.png", arnold_watermark)


In [35]:
arnold_watermark = cv2.imread("/content/arnold_watermark.png", cv2.IMREAD_GRAYSCALE)
watermark = invert_arnold_transform(arnold_watermark, 10)
cv2.imwrite("watermark.png", watermark)

In [36]:
def embed(filename, watermarkname, outfilename):
  img = cv2.imread(filename)
  print("Image size: {}".format(img.shape))
  watermark = cv2.imread(watermarkname, cv2.IMREAD_GRAYSCALE)
  print("Watermark size: {}".format(watermark.shape))

  out_img = img.copy()  # Tạo ảnh đầu ra kích thước như ảnh gốc

  arnold_watermark = arnold_transform(watermark, 10)  # Biến đổi arnold cho dấu thủy vân
  w = arnold_watermark.flatten()  # Làm phẳng dấu thủy vân 1D
  w = np.array(w/w.max(),dtype=np.uint8)
  print(w[:10])
  cv2.imwrite("arnold_watermark.png", arnold_watermark) # Lưu lại dấu thủy vân sau biến đổi arnold

  # Quét ảnh theo BLOCKSIZE = 4x4
  i = 0
  accepted = 0
  not_accepted = 0
  with open("cholesky.txt", "w", encoding="utf-8") as f:
    for y in range(0, img.shape[0], BLOCKSIZE):
      for x in range(0, img.shape[1], BLOCKSIZE):
        block = img[y:y+BLOCKSIZE, x:x+BLOCKSIZE, 0].astype(np.uint8)
        #cholesky sẽ bị lỗi nếu kích thước các khối ảnh là khác nhau, điều này xảy ra khi kích thước ảnh không chia hết cho BLOCKSIZE
        try:
          cholesky(block)
        except:
          not_accepted += 1 #Nếu phát sinh lỗi thì bỏ qua
          continue
        if cholesky(block) is False: #Kiểm tra điều kiện cholesky
          not_accepted += 1
          continue # Nếu không thỏa điều kiện định thức thì bỏ qua
        else: # Nếu thỏa thì xử lý phân tích cholesky
          A = block
          try:
            L = np.linalg.cholesky(A)
          except:
            not_accepted += 1 #Nếu phát sinh lỗi thì bỏ qua
            continue
          L = np.linalg.cholesky(A)
          LT = np.transpose(L)
          if i == len(w):
            i = 0
          z = L[0, 0] - L[1, 0]
          value = w[i]
          #Thực hiện nhúng
          if z >= 0 and value == 1:
            L[0, 0] = L[1, 0] - T #Đã sửa lại vị trí nhúng
          elif z <= 0 and value == 0:
            L[1, 0] = L[0, 0] - T #Đã sửa lại vị trí nhúng
          A = np.dot(L, LT)
          #Thực hiện làm tròn các giá trị
          if value == 1:
            A[0, 0] = math.floor(A[0, 0])
            A[1, 0] = math.ceil(A[1, 0])
          else:
            A[0, 0] = math.ceil(A[0, 0])
            A[1, 0] = math.floor(A[1, 0])
          i += 1
          accepted += 1 #Thỏa mãn phân tích Cholesky
          f.write("{}\n".format(str(A.astype(np.uint8)).replace("\n", "")))
          out_img[y:y+BLOCKSIZE, x:x+BLOCKSIZE, 0] = A.astype(np.uint8)
  cv2.imwrite(outfilename, out_img)
  print("Khối phù hợp: {}".format(accepted))
  print("Khối không phù hợp: {}".format(not_accepted))

In [37]:
embed(filename = "/content/lenna.bmp",
      watermarkname = "/content/w_binary.png",
      outfilename = "/content/embed_watermark.bmp")

Image size: (512, 512, 3)
Watermark size: (32, 32)
32 32
[1 1 1 1 0 1 1 1 0 0]


Khối phù hợp: 2700
Khối không phù hợp: 62836


In [38]:
def extract(filename, outfilename):
  img = cv2.imread(filename)
  print("Image size: {}".format(img.shape))

  w = np.zeros(WM_SIZE**2, dtype=np.uint8)
  cholesky = []
  with open("cholesky.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()
    for line in lines:
      cholesky.append(line.strip("\n"))
  i = 0
  #Thực hiện trích xuất
  for y in range(0, img.shape[0], BLOCKSIZE):
    for x in range(0, img.shape[1], BLOCKSIZE):
        block = img[y:y+BLOCKSIZE, x:x+BLOCKSIZE, 0].astype(np.uint8)
        temp = str(block).replace("\n", "")

        if i < 1024:
          if temp == cholesky[i]:
            value1 = math.sqrt(block[0, 0]) # value1 = L[1, 0] # Đã sửa lại vị trí trích xuất
            value2 = block[1, 0] / math.sqrt(block[0, 0]) # value2 = L[2, 0] #Đã sửa lại vị trí trích xuất
            if value1 > value2: #Đã sửa >= thành >
              w[i] = 0
            else:
              w[i] = 255
            i += 1

  arnold_watermark = w.reshape(32, 32)
  print("Watermark size: {}".format(arnold_watermark.shape))
  watermark = invert_arnold_transform(arnold_watermark, 10)
  cv2.imwrite(outfilename, watermark)

In [39]:
extract(filename = "/content/embed_watermark.bmp",
        outfilename = "/content/extracted_watermark.png")


Image size: (512, 512, 3)
Watermark size: (32, 32)
32 32


In [40]:
def psnr(img1, img2):
    mse = np.mean((img1 - img2)**2)
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20*math.log10(PIXEL_MAX/math.sqrt(mse))

In [42]:
img1 = cv2.imread('/content/avion.bmp')
img2 = cv2.imread('/content/embed_watermark.bmp')
psnr_index = psnr(img1, img2)
print(f"PSNR index: {psnr_index}")

PSNR index: 58.85202103882375


In [45]:
def mssim(img1, img2):
    """
    Tính toán SSIM (Chỉ số tương tự về cấu trúc) cho hình ảnh đầy đủ.
    """
    C1 = (0.01 * 255)**2
    C2 = (0.03 * 255)**2

    img1 = img1.astype(np.uint8)
    img2 = img2.astype(np.uint8)

    mean_img1 = np.mean(img1)
    mean_img2 = np.mean(img2)

    var_img1 = np.var(img1)
    var_img2 = np.var(img2)

    std_img1 = np.sqrt(var_img1)
    std_img2 = np.sqrt(var_img2)

    cov = np.mean((img1 - mean_img1) * (img2 - mean_img2))

    numerator = (2 * mean_img1 * mean_img2 + C1) * (2 * cov + C2)
    denominator = (mean_img1**2 + mean_img2**2 + C1) * (var_img1 + var_img2 + C2)

    ssim_index = numerator / denominator
    return ssim_index

In [46]:
img1 = cv2.imread("/content/w_binary.png", cv2.IMREAD_GRAYSCALE)
img2 = cv2.imread('/content/extracted_watermark.png', cv2.IMREAD_GRAYSCALE)
ssim_index = mssim(img1, img2)
print(f"SSIM index: {ssim_index}")

SSIM index: 1.0


In [47]:
def nc(img1, img2):
    # Chuyển đổi kiểu dữ liệu sang float để tính toán
    image1 = img1.astype(np.float64)
    image2 = img2.astype(np.float64)

    # Tính giá trị trung bình của mỗi hình ảnh
    mean_image1 = np.mean(image1)
    mean_image2 = np.mean(image2)

    # Chuẩn hóa hình ảnh bằng cách trừ đi giá trị trung bình
    norm_image1 = image1 - mean_image1
    norm_image2 = image2 - mean_image2

    # Tính tử số và mẫu số của công thức NC
    numerator = np.sum(norm_image1 * norm_image2)
    denominator = np.sqrt(np.sum(norm_image1**2) * np.sum(norm_image2**2))

    # Tránh chia cho 0
    if denominator == 0:
        return 0

    # Tính và trả về chỉ số NC
    nc_index = numerator / denominator
    return nc_index

In [48]:
img1 = cv2.imread('/content/w_binary.png')
img2 = cv2.imread('/content/extracted_watermark.png')
nc_index = nc(img1, img2)
print(f"NC index: {nc_index}")

NC index: 1.0
